# "The Eye Of The Typer" Dataset

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
import re
from pathlib import Path
from typing import Literal, Iterator
from datetime import datetime, timedelta
from pprint import pprint

import polars as pl
import rerun as rr
import cv2 as cv

from eott_dataset import *
from eott_dataset.rerun import *

In [ ]:
# %env set EOTT_DATASET_PATH
df = read_participant_characteristics()
df

In [ ]:
participants = [Participant.create(**row) for row in df.iter_rows(named=True)]

print("Participants:", len(participants), end="\n\n")

In [ ]:
p = participants[0]
pprint(p)

In [ ]:
study = Study.DOT_TEST

recording_id = f"{p.participant_id}/{study}"

rr.init("EOTT", recording_id=recording_id, spawn=True)
rr.log(
    "participant",
    rr.TextDocument(
        "\n".join(f"[{key}]\n{value}\n" for key, value in p.to_dict().items())
    ),
    timeless=True,
)

screen_cap: cv.VideoCapture | None = None
webcam_cap: cv.VideoCapture | None = None
webcam_path: Path | None = None

screen_size: tuple[int, int] = (p.display_width, p.display_height)
screen_factor = 2 if p.setting is Setting.LAPTOP else 1

webcam_scale = .5
screen_scale = .5

if p.screen_recording is not None:
    screen_size = tuple(map(lambda v: int(v * screen_scale), screen_size))
    screen_cap = cv.VideoCapture(str(p.screen_recording_path))

rr.log(
    f"screen",
    rr.Boxes2D(
        sizes=[screen_size],
        centers=[tuple(map(lambda v: v * .5, screen_size))],
    ),
    timeless=True,
)

rr.log(
    "participant/pupil/left/tobii",
    rr.SeriesLine(name="left pupil diameter (tobii)", color=(255, 255, 0), width=1),
    timeless=True,
)
rr.log(
    "participant/pupil/right/tobii",
    rr.SeriesLine(name="right pupil diameter (tobii)", color=(255, 0, 255), width=1),
    timeless=True,
)

timeline_df = get_timeline(p).filter(pl.col("study") == study.value)

frame_index: int
webcam_index: int | None
study_name: str | None
offset_time: timedelta
source_name: Source
for log_index, (
    frame_index,
    webcam_index,
    study_name,
    source_name,
    offset_time,
) in enumerate(timeline_df.iter_rows()):
    study_name = Study(study_name) if study_name is not None else None

    rr.set_time_sequence("log_index", log_index)
    rr.set_time_seconds("capture_time", offset_time.total_seconds())

    timeline_name = f"webcam-{webcam_index}" if source_name == "webcam" else source_name
    rr.set_time_sequence(f"{timeline_name}_index", frame_index)

    match source_name:
        case "tobii":
            rerun_log_tobii(
                p.tobii_gaze_predictions.row(frame_index, named=True),
                screen=screen_size,
            )

        case "screen" if screen_cap is not None:
            assert p.screen_recording is not None
            assert p.screen_offset is not None
            screen_cap = rerun_log_screen(
                screen_cap, position=frame_index, size=screen_size
            )

        case "webcam" if webcam_index is not None and study_name is not None:
            paths = p.get_webcam_video_paths(study=study_name, index=webcam_index)

            if len(paths) > 0 and (webcam_path != paths[0] or webcam_cap is None):
                webcam_path = paths[0]
                webcam_cap = cv.VideoCapture(str(webcam_path.absolute()))

            webcam_cap = rerun_log_webcam(webcam_cap, name=None, scale=0.5)

        case "log":
            rerun_log_user(p.user_interaction_logs.row(frame_index, named=True), scale=screen_scale)

# end of logging
if screen_cap is not None:
    screen_cap.release()

if webcam_cap is not None:
    webcam_cap.release()